In [1]:
trainSize = "2000examples_400x400complex.mat" 

dataset="test" #"val", "train" "test"

model_name_bb = '/home/elhamod/projects/jiesRepo/Cophy-PGNN/eval_saved/electromagnetic/models/-0x544ff2c934fa5926.pt' #DNN
model_name_cophy = '/home/elhamod/projects/jiesRepo/Cophy-PGNN/eval_saved/electromagnetic/models/0x49583cf87637eb8.pt' #Cophy-PGNN
model_name_analogue = '/home/elhamod/projects/jiesRepo/Cophy-PGNN/eval_saved/electromagnetic/models/0x575808717bbd1e5b.pt' #PGNN-Analogue

In [2]:
# NN parameters
depth=2
hidden=100
out = 804
in_ = 10

smallest_eigen_val = False

In [3]:
import time
import torch
import numpy as np
import pandas as pd

import sys
sys.path.append('../../scripts')
sys.path.append('../')

from util.solvers import *
from util.utils import eval_all
from data_loader_electromagnetic import DatasetLoader
import torch.nn.functional as f

sys.path.append('Cophy-PGNN/')
from DNN import DNN

import os
sys.path.append(os.path.abspath('../../loss_surface_vis'))
from helper_electromagnetic import getOverlapIntegral, getEigError

In [4]:
torch.cuda.set_device(2)
device = torch.device('cuda:2')

In [5]:
class Electromagnetic_dataset(object):
    def __init__(self, dataset="test"):
        data = DatasetLoader(
            train_size=trainSize, #,
            data_path='//raid/elhamod/Eignedecomposition/Electromagnetic/electromagnetic_dualphase', 
        )
        
        # normalization
        data.normalization(x=False, y=False)
        
        self.input_=data.X_test if dataset=="test" else (data.X_val if dataset=="val" else data.X_train)
        self.true_=data.y_test if dataset=="test" else (data.y_val if dataset=="val" else data.y_train)
        H=data.H_test_origin if dataset=="test" else (data.H_val_origin if dataset=="val" else data.H_train_origin)
        H_complex_width = int(H.shape[1]/2)
        
        H_real = H[:, :H_complex_width, :H_complex_width]
        H_img = H[:, H_complex_width:, :H_complex_width]
        H_complex = torch.stack([H_real, H_img]).permute(1, 2, 3, 0).contiguous().numpy()
        self.H = H_complex[:, :, :, 0] + 1j*H_complex[:, :, :, 1]
        self.H_expanded = H

        self.data = data
        self.cnt=0
       
    def count(self):
        return self.H.shape[0]
        
    def get(self):
        cnt = self.cnt
        self.cnt = self.cnt+1
        return self.input_[cnt, :], self.H[cnt,:,:]

In [6]:
dataset = Electromagnetic_dataset(dataset = dataset)

../../scripts/data_loader_electromagnetic.py:134: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.y_train_origin = torch.tensor(self.y_train).float()
../../scripts/data_loader_electromagnetic.py:135: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.y_val_origin = torch.tensor(self.y_val).float()
../../scripts/data_loader_electromagnetic.py:136: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.y_test_origin = torch.tensor(self.y_test).float()


In [7]:
# ======================= PyTorch ========================  


class CophyPGNN(EigenSolver):
    def __init__(self, model_name):
        super().__init__()
        
        with torch.no_grad(): 
            model_gpu = DNN(in_, hidden, out, depth, act=torch.nn.Tanh)
            model_gpu.load_state_dict(torch.load(model_name, map_location=device)) 
            model_gpu = model_gpu.double().cuda()
            model_gpu.eval()
            
        self.solvers = {
            'cophy-pgnn_cuda.'+model_name: model_gpu,
        }
    
    def run(self, m, sort=True):
        m,h = m

        results = self.init_dict()
        runtime = self.init_dict()
        errors = self.init_dict()
        
        for name in self.solvers:
            solver = self.solvers[name]
            elapse = 0

                        
            if 'cuda' in name:
                device = torch.device('cuda')
            else:
                device = torch.device('cpu')
            
            
            # solve and record runtime
            m = torch.tensor(m, dtype=torch.double).cuda(device)
            start = time.process_time()             # start
            with torch.no_grad():
                result = solver(m).detach().cpu().numpy()
            elapse += time.process_time() - start   # end
        
            val = result[-2:]
            vec = result[:-2]

            # post-exec sort results
            val = val[0] + 1j*val[1]
            
            V_complex_width = int(vec.shape[0]/2)
            vec = vec[:V_complex_width] + 1j*vec[V_complex_width:]
            
            # record
            runtime[name] = elapse
            results[name] = (val, vec)
            
            errors[name] = np.linalg.norm(h @ vec - 2*2*np.pi*np.pi*val*val * vec)/np.linalg.norm(h @ vec)
            
            
            n = np.linalg.norm(vec)
            
        return runtime, n, errors, results
    
    


In [8]:
solvers = [CophyPGNN(model_name_bb), CophyPGNN(model_name_cophy), CophyPGNN(model_name_analogue), NumpySolvers(smallest_eigen = smallest_eigen_val),  ScipySolvers(smallest_eigen = smallest_eigen_val)]
df, norms, predicted_vecs_all, predicted_vals_all= eval_all(dataset, solvers)

100%|██████████| 1629/1629 [29:56<00:00,  1.10s/it]


In [9]:
overlap_int = {}
eigen_error= {}

true_reformatted = dataset.true_[:, :-2]

for solver in predicted_vecs_all:
    predicted_vecs_all_real = predicted_vecs_all[solver].real
    predicted_vecs_all_imag = predicted_vecs_all[solver].imag
    prediction_reformatted = torch.cat((torch.tensor(predicted_vecs_all_real), torch.tensor(predicted_vecs_all_imag)), 1)
    
    predicted_vals_all_real = predicted_vals_all[solver].real
    predicted_vals_all_imag = predicted_vals_all[solver].imag
    prediction_reformatted_vals = torch.cat((torch.tensor(predicted_vals_all_real).view(-1,1), torch.tensor(predicted_vals_all_imag).view(-1,1)), 1)
    
    true_reformatted_normalized = f.normalize(true_reformatted, p=2, dim=1)
    prediction_reformatted_normalized = f.normalize(prediction_reformatted, p=2, dim=1)
    
    average_overlapInt = sum(getOverlapIntegral(prediction_reformatted_normalized, true_reformatted_normalized))/predicted_vecs_all[solver].shape[0]
    overlap_int[solver] = average_overlapInt
    
df2 = pd.DataFrame.from_dict(overlap_int, orient='index')
df['overlap_int'] = df2

In [10]:
df

,avg time,avg eigen error,overlap_int
cophy-pgnn_cuda./home/elhamod/projects/jiesRepo/Cophy-PGNN/eval_saved/electromagnetic/models/-0x544ff2c934fa5926.pt,0.041552,7.205025e-01,0.579846
cophy-pgnn_cuda./home/elhamod/projects/jiesRepo/Cophy-PGNN/eval_saved/electromagnetic/models/0x49583cf87637eb8.pt,0.032158,1.308940e-01,0.645425
cophy-pgnn_cuda./home/elhamod/projects/jiesRepo/Cophy-PGNN/eval_saved/electromagnetic/models/0x575808717bbd1e5b.pt,0.030694,1.980061e+00,0.616318
numpy_eig,35.819889,1.465940e-07,0.999999
scipy_eig,33.707246,2.620192e-05,0.982211


In [11]:
df.to_csv("results/stats.csv")